In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

base_model = "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForSequenceClassification.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16)

model = model.eval()

In [2]:
tokens = tokenizer(["Fuck this economy. I hate aig and their non loan given asses"], return_tensors='pt',
                           padding=True)
output = model(**tokens)

output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
output.argmax()

tensor(1)

In [3]:
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset
from tqdm import tqdm
import torch
import numpy as np

def map_output(x):
    dic = {0:0, 1:2, 2:1}
    return dic[x]

def test_fiqa(model, tokenizer, batch_size=8):
    dataset = load_dataset('pauri32/fiqa-2018')
    dataset = dataset["test"]
    dataset = dataset.to_pandas()

    dataset["target"] = dataset['label']

    dataset = dataset[['sentence', 'target']]
    dataset.columns = ['input', 'target']

    # print example
    print(f"\n\nPrompt example:\n{dataset['input'][1]}\n\n")

    context = dataset['input'].tolist()
    total_steps = dataset.shape[0]//batch_size + 1
    print(
        f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text = []

    for i in tqdm(range(total_steps)):
        tmp_context = context[i * batch_size:(i+1) * batch_size]

        tokenizer.pad_token = "[PAD]"

        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True)

        output = model(**tokens)
        output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
        out_text.append(output.detach().numpy())
        torch.cuda.empty_cache()

    out_text = [item for sublist in out_text for item in sublist]
    dataset["out_text"] = out_text
    dataset["new_out"] = dataset["out_text"].apply(np.argmax).apply(map_output)

    acc = accuracy_score(dataset["target"], dataset["new_out"])
    f1_macro = f1_score(dataset["target"], dataset["new_out"], average="macro")
    f1_micro = f1_score(dataset["target"], dataset["new_out"], average="micro")
    f1_weighted = f1_score(
        dataset["target"], dataset["new_out"], average="weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    return dataset


dataset = test_fiqa(model, tokenizer)
dataset



Prompt example:
@gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?


Total len: 150. Batchsize: 8. Total steps: 19


100%|██████████| 19/19 [00:01<00:00, 18.17it/s]

Acc: 0.4066666666666667. F1 macro: 0.3735183751740043. F1 micro: 0.4066666666666667. F1 weighted (BloombergGPT): 0.3735183751740043. 


,input,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna giv...,0,"[0.72789717, 0.009888096, 0.26221472]",0
1,@gakrum nice chart shows distinctive down chan...,2,"[0.41241357, 0.09974262, 0.48784387]",1
2,Japan's Asahi to submit bid next week for SABM...,0,"[0.26295966, 0.14501196, 0.5920284]",1
3,"Tesla Motors recalls 2,700 Model X SUVs $TSLA ...",2,"[0.42264017, 0.119706936, 0.4576529]",1
4,CRH's concrete bid for Holcim Lafarge assets,0,"[0.24879104, 0.091215596, 0.65999335]",1
...,...,...,...,...
145,Intertek swings to ÃÂ£347 mln loss on oil's s...,2,"[0.12549062, 0.228127, 0.64638233]",1
146,RT @jan $ARNA Don't think buyout rumor strong ...,1,"[0.21853669, 0.27272964, 0.50873363]",1
147,Barclays appoints JPMorgan's Paul Compton as n...,1,"[0.23756759, 0.4300743, 0.33235818]",2
148,$AAPL Now I'm glad I got stopped out of my $11...,1,"[0.11438672, 0.15592879, 0.72968453]",1


In [5]:
path = '../../results/financial/finbert.csv'

In [6]:
dataset.to_csv(path)

In [7]:
import pandas as pd

df = pd.read_csv(path, index_col=0)

true, pred = df["target"], df["new_out"]


import sys
sys.path.append('../../')
from metrics import metrics

metrics(true, pred)

Precision: 0.44, Recall: 0.407, F1: 0.374, Accuracy: 0.407
